<a href="https://colab.research.google.com/github/MuneebBaig71/Chatbot/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

In [2]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [29]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
userdata.get('Pinecone_DB')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['Pinecone_DB'] = userdata.get('Pinecone_DB')

In [30]:
# Ensure your VertexAI credentials are configured

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")

In [32]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

api_key = os.environ.get('Pinecone_DB')
pc = Pinecone(api_key= api_key, environment='us-east-1')
index_name = "quickstart2"

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [33]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = PyPDFLoader("Human_Spirit.pdf")  # Replace with your file
documents = loader.load()


# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'Human_Spirit.pdf', 'page': 0}, page_content='The Resilience of the Human Spirit\nThroughout  history,  the  resilience  of  the  human  spirit  has  been  a  recurring  theme,\nunderscoring  humanity’s  ability  to  endure,  adapt,  and  overcome  adversity.  This\nremarkable capacity is not limited to extraordinary individuals but is an intrinsic part of\nour shared human experience. From natural disasters to personal setbacks, the human\nspirit  continually  demonstrates  an  unwavering  determination  to  persevere,  often'), Document(metadata={'source': 'Human_Spirit.pdf', 'page': 0}, page_content='emerging stronger and more enlightened.\nOne  of  the  most  profound  examples  of  human  resilience  is  seen  in  the  aftermath  of\nnatural  calamities.  Earthquakes,  hurricanes,  and  tsunamis  have  devastated  entire\ncommunities, leaving behind immense destruction and despair. Yet, in these moments\nof  darkness,  humanity’s  collective  resilien

In [34]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content))
    metadata = {"text": doc.page_content}
    # Upsert the vector with a unique ID
    index.upsert(vectors=[(doc_id, vector,metadata)])

100%|██████████| 9/9 [00:02<00:00,  3.22it/s]


In [35]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

In [36]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever

)

In [37]:
while True:
  user_input = input("Ask something: ")
  if user_input == "quit":
    break
  else:
    response = qa_chain.run(user_input)
    print(response)

Ask something: hi
You still haven't provided an original answer to refine.  The new context you've added describes the resilience of people facing adversity, using examples of cultural striving and scientific innovation during the COVID-19 pandemic.  This is helpful thematic information, but I need the *original answer* to refine it.  Please provide the original answer you want me to improve.

Ask something: generate me a summary
The text celebrates human resilience as a defining characteristic, highlighting its role in driving progress and achieving remarkable feats throughout history.  It uses examples of overcoming both personal struggles and large-scale crises (implicitly referencing natural disasters and global events like the COVID-19 pandemic) to illustrate this enduring quality.  The conclusion emphasizes that this resilience isn't simply a reaction to hardship, but an active force propelling humanity towards a better future, showcasing our collective strength and determination